<a href="https://colab.research.google.com/github/macapagithub/churn-prediction/blob/main/notebooks/exp_enviroment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import userdata
path_data = userdata.get('path_dataset')

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
print(path_data)

/content/drive/MyDrive/prueba/dataset.csv


In [ ]:
pd.read_csv(path_data, sep=';', decimal=",").head(2)

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,...,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,...,0.0,N,U,U,U,U,U,Y,361.0,1000001
1,57.4925,482.75,37.425,0.2475,22.75,9.1,9.1,0.0,0.0,532.25,...,0.0,Z,U,U,U,U,U,Y,240.0,1000002


In [ ]:
df = pd.read_csv(path_data, sep=';', decimal=",")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import (StandardScaler,
                                   OneHotEncoder,
                                   MinMaxScaler,
                                   Normalizer,
                                   LabelEncoder)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['churn', 'infobase'], axis=1),
                                                    df['churn'],
                                                    test_size=0.3,
                                                    random_state=12)

In [ ]:
X_train_, X_eval, y_train_, y_eval = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=12)

In [ ]:
num_variables = [var for var in X_train.columns if X_train[var].dtypes != object]
cat_variables = [var for var in X_train.columns if X_train[var].dtypes == object]

In [ ]:
numeric_trans = Pipeline(steps=[
                              ("imputer", SimpleImputer(strategy="mean")),
                              ("scaler", StandardScaler())
                              # ("normal", Normalizer())
                              ])

categorical_trans = Pipeline(steps=[
           ("imputer", SimpleImputer(strategy="constant")),
           ("oneHotE", OneHotEncoder(handle_unknown="ignore"))
          #  ('ordinal', OrdinalEncoder())
           ])

transformation = ColumnTransformer(transformers=[
        ("num", numeric_trans, num_variables),
        ("cat", categorical_trans, cat_variables),
    ])

In [ ]:
%%time
model = RandomForestClassifier(random_state=10)
clf = Pipeline(steps=[
              ("transformer", transformation),
              ("classifier", model)]
)

clf.fit(X_train_, y_train_)
print("Model score: %.3f" % clf.score(X_eval, y_eval))

Model score: 0.614
CPU times: user 31.6 s, sys: 264 ms, total: 31.9 s
Wall time: 34.9 s


In [ ]:
%%time
model = GradientBoostingClassifier(random_state=10)
clf = Pipeline(steps=[
              ("transformer", transformation),
              ("classifier", model)]
)

clf.fit(X_train_, y_train_)
print("Model score: %.4f" % clf.score(X_eval, y_eval))

Model score: 0.6225
CPU times: user 1min 53s, sys: 455 ms, total: 1min 54s
Wall time: 1min 54s


In [ ]:
pip install xgboost

In [ ]:
%%time
from xgboost import XGBClassifier


model = XGBClassifier()
clf = Pipeline(steps=[
              ("transformer", transformation),
              ("classifier", model)]
)

clf.fit(X_train_, y_train_)
print("Model score: %.4f" % clf.score(X_eval, y_eval))

Model score: 0.7729
CPU times: user 9.37 s, sys: 287 ms, total: 9.66 s
Wall time: 6.07 s


In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix
# def get_model_performance_test_set():
#     y_pred = clf.predict(X_eval)



#     fig, ax = plt.subplots()
#     fig.set_figheight(8)
#     fig.set_figwidth(8)

#     mat = confusion_matrix(y_eval, y_pred)
#     sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)#,
#             # xticklabels=y.categories, yticklabels=y.categories)


#     ax.set_xlabel('Model prediction')
#     ax.set_ylabel('Real prediction')
#     ax.set_title('Behavior of model prediction')
#     fig.savefig('prediction_behavior.png')

# get_model_performance_test_set()

In [ ]:
# from sklearn.inspection import permutation_importance

# feature_importance = clas.feature_importances_
# sorted_idx = np.argsort(feature_importance)
# pos = np.arange(sorted_idx.shape[0]) + 0.5
# fig = plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.barh(pos, feature_importance[sorted_idx], align="center")
# # plt.yticks(pos, np.array(diabetes.feature_names)[sorted_idx])
# plt.title("Feature Importance")

# result = permutation_importance(
#     clas, X_eval_proc, y_eval_proc, n_repeats=10, random_state=42, n_jobs=2
# )
# sorted_idx = result.importances_mean.argsort()
# plt.subplot(1, 2, 2)
# plt.boxplot(
#     result.importances[sorted_idx].T,
#     vert=False,
#     # labels=np.array(diabetes.feature_names)[sorted_idx],
# )
# plt.title("Permutation Importance (test set)")
# fig.tight_layout()
# plt.show()

In [ ]:
# sorted_idx = result.importances_mean.argsort()

# fig = plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.barh(pos[209:], feature_importance[sorted_idx[209:]], align="center")
# plt.yticks(pos[209:], np.array(X_.columns)[sorted_idx[209:]])
# plt.title("Feature Importance")


# plt.subplot(1, 2, 2)
# plt.boxplot(
#     result.importances[sorted_idx[209:]].T,
#     vert=False,
#     labels=np.array(X_.columns)[sorted_idx[209:]],
# )
# plt.title("Permutation Importance (test set)")
# fig.tight_layout()
# plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

xgb_clf = XGBClassifier()

clf = Pipeline(steps=[
              ("transformer", transformation),
              ("classifier", xgb_clf)]
)

param_dist = {
    'classifier__n_estimators': np.arange(50, 300, 50),
    'classifier__learning_rate': np.linspace(0.01, 0.3, 10),
    'classifier__max_depth': np.arange(3, 10),
    'classifier__min_child_weight': np.arange(1, 7),
    'classifier__subsample': np.linspace(0.5, 1.0, 5),
    'classifier__colsample_bytree': np.linspace(0.5, 1.0, 5),
    'classifier__gamma': np.linspace(0, 0.5, 5)
}

random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=20,
    scoring='f1',
    cv=3,
    verbose=3,
    random_state=12,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('transformer',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['rev_Mean',
                                                                                'mou_Mean',
                                                                                'totmrc_Mean',
                                                                                'da_Mean',
                                                                                'ovrmou_Mean',
                                                                                'ovrrev_Mean',
                                                                                'vceovr_Mean',
                                                                                'datovr_Mean',
                                                                                'roam_Mean',
                                                                                'change_mou',
                                                                                'change_rev',
                                                                                'drop_vce_Mean',
                                                                                'drop_dat_Mean'...
                                        'classifier__learning_rate': array([0.01      , 0.04222222, 0.07444444, 0.10666667, 0.13888889,
       0.17111111, 0.20333333, 0.23555556, 0.26777778, 0.3       ]),
                                        'classifier__max_depth': array([3, 4, 5, 6, 7, 8, 9]),
                                        'classifier__min_child_weight': array([1, 2, 3, 4, 5, 6]),
                                        'classifier__n_estimators': array([ 50, 100, 150, 200, 250]),
                                        'classifier__subsample': array([0.5  , 0.625, 0.75 , 0.875, 1.   ])},
                   random_state=12, scoring='f1', verbose=3)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

best_model = random_search.best_estimator_

y_pred = best_model.predict(X_eval)
accuracy = accuracy_score(y_eval, y_pred)
f1_score = f1_score(y_eval, y_pred)

print(f"Mejores hiperparámetros: {random_search.best_params_}")
print(f"Precisión en test: {accuracy:.4f}")
print(f"F1 score en test: {f1_score:.4f}")

Mejores hiperparámetros: {'classifier__subsample': 0.75, 'classifier__n_estimators': 250, 'classifier__min_child_weight': 5, 'classifier__max_depth': 8, 'classifier__learning_rate': 0.01, 'classifier__gamma': 0.125, 'classifier__colsample_bytree': 0.625}
Precisión en test: 0.6926
F1 score en test: 0.6997


### Code to review

In [ ]:


param_grid = [
    # GradientBoostingClassifier
    {
        'classifier': [GradientBoostingClassifier(random_state=10)],
        'classifier__n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600],
        'classifier__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
        'classifier__max_depth': [1, 3, 5, 8]
    },
    # RandomForestClassifier
    {
        'preprocesser__num_transform__imputer__strategy': ['mean', 'median'],
        'classifier': [RandomForestClassifier(random_state=10)],
        'classifier__n_estimators': [10, 100, 1000]
    }
]

model = GradientBoostingClassifier(random_state=10)
pipeline = Pipeline(steps=[
              ("transformer", transformation),
              ('classifier', 'passthrough')]
              # ('feature', SelectFromModel(estimator=LogisticRegression())),
)

In [ ]:
%%time
random_search = RandomizedSearchCV(
    pipeline,
    n_iter=20,
    param_distributions=param_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

# Ajustar el modelo a los datos
random_search.fit(X_train_, y_train_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


KeyboardInterrupt: 

In [ ]:
random_search.best_score_

In [ ]:
%%time
best_model = random_search.best_estimator_
print("Mejor modelo:", type(best_model.named_steps['classifier']).__name__)
print("Mejores parámetros:", random_search.best_params_)
print(f"Mejor puntuación:, {random_search.best_score_:.3f}")

In [ ]:
print("Best params:")
print(random_search.best_params_)

print(f"Internal CV score: {random_search.best_score_:.3f}")